## Hidden Markov Model
This notebook is an implementation of Parts of Speech (POS) tagging task using Hidden Markov Models.

The dataset used for training is:  https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz 

Format of the training file:

&lt;Word/Token&gt;   &lt;POS-TAG&gt;  &lt;Chunking Tag&gt;

**Note: For the implementation of the POS tagging we are only using the first two columns of the training file.

#### Overview:

The implementation of HMM is done using a dynamic programming algorithm named Viterbi. In order to improve the accuracy of POS tagging futher the vainila verison of Viterbi is incorporated to futher process the dataset to mark the unknown state based on the most likely POS-tagging for the particular word. Also, the vocabulary is formed of the training data.

Dataset is split into 80-20 ratio with 80% as training data and 20% as test data to montior the performance of the model.

#### Accuracy:   

| Algo      | Limited Test Dataset | Full test dataset |
| ----------- | ----------- | ----------- |
| Vainilla Viterbi  | 87.77% | 90.79%      |
|  Regex Viterbi   | 92.13% | 94.98%   |


**Note: Due to large computation nature of HMM the model takes significant time to run approx 7-8 hours to complete run on 20% of the test data. 

## Getting started
#### Install dependencies as:
- [ ] Python (>=3)
- [ ] re
- [ ] numpy
- [ ] scikit-learn
- [ ] nltk (optional, can be used to pre-process the dataset) :: In this notebook the preprocessing is done manually.

#### Files needed: 
- [ ] train.txt 
- [ ] test.txt

train and test file can be found in the sub-directory.


#### Sample Output:

[('Dollar', 'NNP'), (':', ':'), ('142.75', 'CD'), ('yen', 'NN'), (',', ','), ('up', 'IN'), ('0.95', 'CD'), (';', ':'), ('1.8667', 'CD'), ('marks', 'NNS'), (',', ','), ('off', 'IN'), ('0.0018', '``'), ('.', '``'), ('Amex', 'NNP'), ('issues', 'NNS'), ('with', 'IN'), ('big', 'JJ'), ('percentage', 'NN')]













In [4]:
import nltk
import re, random
from sklearn.model_selection import train_test_split
import time
import numpy as np
import pandas as pd

#### Data Preprocessing:

In [8]:
class data_preprocessing:

  def __init__(self):
    self.train_labelled_data = []
    self.test_labelled_data = []
    self.tags = {}
    self.vocab = {}
    self.test_set = []
    self.train_set = []
    self.create_tags_vocab()

  def file_processing(self,file_name = "train.txt"):

    # Reading the dataset file
    with open(file_name, 'r') as file:
      data = file.read()
    print("Train Corpus Sample:")
    print(f"{data[0:50]} \n")

    # Extracting the sentences from data and creating a sentences list []
    sentences = data.strip().split('\n\n')
    print("Sentences Sample:")
    print(sentences[:2])

    processed_sentences = []

    for sentence in sentences:
      sent = []
      # Split the sentence into individual lines (tokens and tags)
      lines = sentence.strip().split('\n')
      # Extract the tokens and tags from each line
      tokens_tags = [line.split() for line in lines]
      # Extract the tokens and tags into separate lists
      for token_tag in tokens_tags:
        token, tag = token_tag[0], token_tag[1]
        sent.append((token, tag))
      processed_sentences.append(sent)
    
    return processed_sentences

  def split_train_test(self):
    processed_sentences = self.file_processing()

    # Showing sample data
    for token_tag in processed_sentences[:2]:
      for tuple in token_tag:
        print(tuple)
    print("\n")

    # split data into training and validation set in the ratio 80:20
    self.train_set,self.test_set = train_test_split(processed_sentences,train_size=0.80,test_size=0.20,random_state = 100)

    # create list of train and test tagged words
    self.train_labelled_data = [tuple for token_tag in self.train_set for tuple in token_tag]
    self.test_labelled_data = [tuple for token_tag in self.test_set for tuple in token_tag]
    print(f"Length of the training labels: \t{len(self.train_labelled_data)}")
    # check some of the train labelled data.
    print(f'Training labelled words \t {self.train_labelled_data[:5]}')

    print(f"Length of the testing labels: \t{len(self.test_labelled_data)}")
    # check some of the test labelled data.
    print(f'Testing labelled words \t {self.test_labelled_data[:5]}')

    print("\n")

  def create_tags_vocab(self):
    self.split_train_test()
    #use set datatype to check how many unique tags are present in training data
    self.tags = {tag for word,tag in self.train_labelled_data}
    print(f"Length of tags in training labelled data: \t{len(self.tags)}")
    print(self.tags)
    print("\n")

    # check total words in vocabulary
    self.vocab = {word for word,tag in self.train_labelled_data}
    print(f"Length of Vocab in training labelled data: \t{len(self.vocab)}")
    print(self.vocab)
    print("\n")


In [9]:
processed_data = data_preprocessing()
train_labelled_data = processed_data.train_labelled_data
test_set, train_set, vocab, tags = processed_data.test_set, processed_data.train_set, processed_data.vocab, processed_data.tags

Train Corpus Sample:
Confidence NN B-NP
in IN B-PP
the DT B-NP
pound NN 

Sentences Sample:
["Confidence NN B-NP\nin IN B-PP\nthe DT B-NP\npound NN I-NP\nis VBZ B-VP\nwidely RB I-VP\nexpected VBN I-VP\nto TO I-VP\ntake VB I-VP\nanother DT B-NP\nsharp JJ I-NP\ndive NN I-NP\nif IN B-SBAR\ntrade NN B-NP\nfigures NNS I-NP\nfor IN B-PP\nSeptember NNP B-NP\n, , O\ndue JJ B-ADJP\nfor IN B-PP\nrelease NN B-NP\ntomorrow NN B-NP\n, , O\nfail VB B-VP\nto TO I-VP\nshow VB I-VP\na DT B-NP\nsubstantial JJ I-NP\nimprovement NN I-NP\nfrom IN B-PP\nJuly NNP B-NP\nand CC I-NP\nAugust NNP I-NP\n's POS B-NP\nnear-record JJ I-NP\ndeficits NNS I-NP\n. . O", "Chancellor NNP O\nof IN B-PP\nthe DT B-NP\nExchequer NNP I-NP\nNigel NNP B-NP\nLawson NNP I-NP\n's POS B-NP\nrestated VBN I-NP\ncommitment NN I-NP\nto TO B-PP\na DT B-NP\nfirm NN I-NP\nmonetary JJ I-NP\npolicy NN I-NP\nhas VBZ B-VP\nhelped VBN I-VP\nto TO I-VP\nprevent VB I-VP\na DT B-NP\nfreefall NN I-NP\nin IN B-PP\nsterling NN B-NP\nover IN B-PP\nthe

#### Defining the model 

In [10]:
class HMM:

  def __init__(self, train_bag):
    self.train_bag = train_bag
   
  # Emission Probability
  def emission_prob(self, word, tag):
      tag_list = [pair for pair in self.train_bag if pair[1]==tag]
      count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
      w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
      #now calculate the total number of times the passed word occurred as the passed tag.
      count_w_given_tag = len(w_given_tag_list)
  
      
      return (count_w_given_tag, count_tag)

  # Transition Probability
  def transition_prob(self, t2, t1):
      tags = [pair[1] for pair in self.train_bag]
      count_t1 = len([t for t in tags if t==t1])
      count_t2_t1 = 0
      for index in range(len(tags)-1):
          if tags[index]==t1 and tags[index+1] == t2:
              count_t2_t1 += 1
      return (count_t2_t1, count_t1)

  # Trainsition Matrix
  def create_transition_matrix(self, tags):
    # creating t x t transition matrix of tags, t= no of tags
    # Matrix(i, j) represents P(jth tag after the ith tag)
    
    tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
    for i, t1 in enumerate(list(tags)):
        for j, t2 in enumerate(list(tags)): 
            tags_matrix[i, j] = self.transition_prob(t2, t1)[0]/self.transition_prob(t2, t1)[1]
    
    print(tags_matrix)
    # convert the matrix to a df for better readability
    #the table is same as the transition table shown in section 3 of article
    tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
    display(tags_df)
    return tags_df

  def Viterbi(self, words, tags_df):
    state = []
    T = list(set([pair[1] for pair in self.train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = self.emission_prob(words[key], tag)[0]/self.emission_prob(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))
  
  def unknown_state(self,word):
    if re.search(r'.*(ing|ed|es|ould)$',word.lower()):
        return 'VBG'
    elif re.search(r'to$',str(word).lower()):
        return 'TO'
    elif re.search(r'^-?[0-9]+(.[0-9]+)?\.*$',str(word).lower()):
        return 'CD'
    elif '*' in word:
        return 'SYM'
    elif re.search(r'.*\'s$',word.lower()): 
        return 'POS'
    elif re.search(r'.*ness$',word.lower()):
        return 'NN'
    elif re.search(r'(The|the|A|a|An|an)$',word):
        return 'DT'
    elif re.search(r'.*able$',word.lower()):
        return 'JJ'
    elif re.search(r'.*ly$',word.lower()):
        return 'RB'
    elif re.search(r'(He|he|She|she|It|it|I|me|Me|You|you|His|his|Her|her|Its|its|my|Your|your|Yours|yours)$',word):
        return 'PRP'
    elif re.search(r'(on|On|at|At|since|Since|For|for|before|Before|till|Till|until|Until|by|By|Beside|beside|under|Under|below|Below|over|Over|above|Above|across|Across|Through|through|Into|into|towards|Towards|onto|Onto|from|From)$',word):
        return 'IN'
    elif re.search(r'',word):
        return 'NNP'
    elif re.search(r'(\'|\"|\.|\(|\)|\?|\[|\]|\:|\;)+',word):
        return '.'
    else:        
        return 'NN'
    
  def regexViterbi(self, words, tags_df):
    state = []
    T = list(set([pair[1] for pair in self.train_bag]))
        
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                    
            # compute emission and state probabilities
            emission_p = self.emission_prob(words[key], tag)[0]/self.emission_prob(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
                
        pmax = max(p)
        if pmax==0.0:
            state_word=self.unknown_state(word)
            state.append(state_word)
            #print(word,':',state_word)
        else:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
    return list(zip(words, state))

  def model_accuracy(self, tagged_seq, input_tagged_words):
    check = [i for i, j in zip(tagged_seq, input_tagged_words) if i == j] 
    accuracy = len(check)/len(tagged_seq)
    print('Viterbi Algorithm Accuracy: ',accuracy*100)

#### Training and Testing on Vainilla Viterbi 

In [5]:
random.seed(1234)
 # choose 10 random sentences and preparing the test_set
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
#converting list of ["words/tokens","pos"] from multiple sentence to one list.
test_run_base = [tup for sent in test_run for tup in sent]
 
# segregating the ["words/tokens"] from the list.
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [6]:
model = HMM(train_labelled_data)
tags_df = model.create_transition_matrix(tags)

start = time.time()
tagged_seq = model.Viterbi(test_tagged_words, tags_df=tags_df)
end = time.time()

difference = end-start
print("Time taken in seconds: ", difference)

[[0.0000000e+00 8.6277731e-02 8.2169269e-04 ... 6.5735416e-03
  8.2169269e-04 0.0000000e+00]
 [2.2446690e-03 1.1676435e-01 7.7732052e-03 ... 2.3693726e-03
  4.1567942e-05 5.2375607e-03]
 [0.0000000e+00 1.5604681e-02 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [4.7393367e-03 2.3696683e-02 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [3.1118587e-02 4.5416318e-02 8.4104287e-03 ... 1.6820858e-03
  0.0000000e+00 6.7283432e-03]]


,``,NN,WDT,EX,UH,JJ,RP,MD,:,),...,WP$,PRP,.,VBP,VB,#,NNP,WP,PDT,''
``,0.000000,0.086278,0.000822,0.022186,0.002465,0.105998,0.000000,0.010682,0.000000,0.000000,...,0.000000,0.217749,0.000000,0.016434,0.021364,0.000000,0.074774,0.006574,0.000822,0.000000
NN,0.002245,0.116764,0.007773,0.000249,0.000000,0.010018,0.000083,0.018040,0.011556,0.001954,...,0.000166,0.005154,0.104793,0.003450,0.001496,0.000042,0.009976,0.002369,0.000042,0.005238
WDT,0.000000,0.015605,0.000000,0.001300,0.000000,0.001300,0.000000,0.131339,0.001300,0.000000,...,0.000000,0.033810,0.000000,0.171651,0.000000,0.000000,0.014304,0.000000,0.000000,0.000000
EX,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.101266,0.006329,0.000000,...,0.000000,0.000000,0.000000,0.151899,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
UH,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
JJ,0.001430,0.459493,0.000477,0.000000,0.000000,0.070816,0.000286,0.000667,0.002955,0.000667,...,0.000095,0.002383,0.025734,0.000286,0.000286,0.000095,0.032692,0.000572,0.000095,0.003812
RP,0.000000,0.085714,0.000000,0.000000,0.000000,0.085714,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
MD,0.005848,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000585,...,0.000000,0.007602,0.002339,0.000000,0.785380,0.000000,0.000000,0.000000,0.000000,0.000000
:,0.059102,0.037825,0.022459,0.003546,0.000000,0.047281,0.000000,0.005910,0.001182,0.000000,...,0.000000,0.043735,0.015366,0.010638,0.011820,0.000000,0.105201,0.005910,0.001182,0.003546
),0.017391,0.039130,0.000000,0.000000,0.000000,0.013043,0.000000,0.021739,0.086957,0.000000,...,0.000000,0.017391,0.139130,0.017391,0.013043,0.000000,0.073913,0.000000,0.000000,0.000000


Time taken in seconds:  126.87295174598694


In [7]:
model.model_accuracy(tagged_seq, test_run_base)

Viterbi Algorithm Accuracy:  87.77292576419214


In [8]:
#Code to test all the test sentences
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words
 
start = time.time()
tagged_seq = model.Viterbi(test_untagged_words, tags_df=tags_df)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)

Time taken in seconds:  25616.368921756744


In [9]:
model.model_accuracy(tagged_seq, test_tagged_words)

Viterbi Algorithm Accuracy:  90.79462390945532


In [2]:
def read_test_file(file_name = "test.txt"):
    with open(file_name, 'r',encoding="utf8") as file:
      data = file.read()
      sentences = data.strip().split('\n\n')
      processed_sentences = []
      for sentence in sentences:
          # Split the sentence into individual lines (tokens and tags)
          lines = sentence.strip().split('\n')
          # append the lines to the processed_sentences list
          processed_sentences.append(lines)
    return processed_sentences

npl_test_set = read_test_file()

In [35]:
# list of test tagged words
test_run_base = [tup for sent in npl_test_set for tup in sent]
print(len(test_run_base))

47377


In [36]:
# sample test data
print(test_run_base[:10])

['Rockwell', 'International', 'Corp.', "'s", 'Tulsa', 'unit', 'said', 'it', 'signed', 'a']


In [37]:
start = time.time()
tagged_seq = model.Viterbi(test_run_base, tags_df=tags_df)
end = time.time()

difference = end-start
print("Time taken in seconds: ", difference)

In [32]:
print(tagged_seq)

[('Dollar', 'NNP'), (':', ':'), ('142.75', 'CD'), ('yen', 'NN'), (',', ','), ('up', 'IN'), ('0.95', 'CD'), (';', ':'), ('1.8667', 'CD'), ('marks', 'NNS'), (',', ','), ('off', 'IN'), ('0.0018', '``'), ('.', '``'), ('Amex', 'NNP'), ('issues', 'NNS'), ('with', 'IN'), ('big', 'JJ'), ('percentage', 'NN'), ('price', 'NN'), ('gains', 'NNS'), ('included', 'VBD'), ('two', 'CD'), ('Eastern', 'NNP'), ('Air', 'NNP'), ('Lines', 'NNPS'), ('preferred', 'JJ'), ('stocks', 'NNS'), (',', ','), ('reacting', 'VBG'), ('to', 'TO'), ('the', 'DT'), ('news', 'NN'), ('about', 'IN'), ('improved', 'VBN'), ('recovery', 'NN'), ('in', 'IN'), ('flight', 'NN'), ('schedules', 'NNS'), ('after', 'IN'), ('the', 'DT'), ('company', 'NN'), ('filed', 'VBD'), ('for', 'IN'), ('bankruptcy', 'NN'), ('protection', 'NN'), ('.', '.'), ('``', '``'), ('We', 'PRP'), ('do', 'VBP'), ("n't", 'RB'), ('like', 'IN'), ('the', 'DT'), ('Congress', 'NNP'), ('-LRB-', '('), ('I', 'PRP'), ('-RRB-', ')'), (',', ','), ("''", "''"), ('says', 'VBZ'), ('

#### Training and Testing on imporved regex Viterbi 

In [11]:
random.seed(1234)
 # choose 10 random sentences and preparing the test_set
rndom = [random.randint(1,len(test_set)) for x in range(10)]
test_run = [test_set[i] for i in rndom]
#converting list of ["words/tokens","pos"] from multiple sentence to one list.
test_run_base = [tup for sent in test_run for tup in sent]
 
# segregating the ["words/tokens"] from the list.
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
model = HMM(train_labelled_data)
tags_df = model.create_transition_matrix(tags)
start = time.time()
tagged_seq = model.regexViterbi(test_tagged_words, tags_df=tags_df)
end = time.time()

difference = end-start
print("Time taken in seconds: ", difference)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.00236967 0.         0.         ... 0.         0.21090047 0.01184834]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01060647 0.00271961 0.         ... 0.         0.0013598  0.04432962]
 [0.01330705 0.00098571 0.         ... 0.         0.00049285 0.00024643]]


,PRP$,WP,UH,$,VBD,WP$,'',EX,FW,WDT,...,:,#,VBP,``,RP,IN,),SYM,VBZ,TO
PRP$,0.000000,0.000000,0.000000,0.004667,0.000667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000667,0.002667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
WP,0.002370,0.000000,0.000000,0.000000,0.258294,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.196682,0.004739,0.000000,0.007109,0.002370,0.000000,0.210900,0.011848
UH,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
VBD,0.025441,0.000743,0.000000,0.017456,0.002043,0.000000,0.000371,0.001300,0.000000,0.000371,...,0.002786,0.000000,0.000186,0.007799,0.001857,0.134633,0.000186,0.000000,0.000743,0.059239
WP$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
'',0.003364,0.001682,0.000841,0.000841,0.078217,0.000000,0.006728,0.000841,0.000000,0.008410,...,0.007569,0.000000,0.004205,0.031119,0.000000,0.138772,0.000841,0.000000,0.137931,0.014298
EX,0.000000,0.000000,0.000000,0.000000,0.183544,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.006329,0.000000,0.151899,0.000000,0.000000,0.000000,0.000000,0.000000,0.487342,0.006329
FW,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.100000,0.000000,...,0.033333,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000
WDT,0.003901,0.000000,0.000000,0.001300,0.244473,0.000000,0.000000,0.001300,0.000000,0.000000,...,0.001300,0.000000,0.171651,0.000000,0.000000,0.011704,0.000000,0.000000,0.288687,0.000000


Time taken in seconds:  140.73422384262085


In [20]:
model.model_accuracy(tagged_seq, test_run_base)

Viterbi Algorithm Accuracy:  92.13973799126637


In [21]:
# model = HMM(train_labelled_data)
# tags_df = model.create_transition_matrix(tags)

#Code to test all the test sentences
test_tagged_words = [tup for sent in test_set for tup in sent]
test_untagged_words = [tup[0] for sent in test_set for tup in sent]
test_untagged_words
 
start = time.time()
tagged_seq = model.Viterbi(test_untagged_words, tags_df=tags_df)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)

model.model_accuracy(tagged_seq, test_tagged_words)

In [ ]:
test_run_base = [tup for sent in npl_test_set for tup in sent]

start = time.time()
tagged_seq = model.Viterbi(test_run_base, tags_df=tags_df)
end = time.time()

difference = end-start
print("Time taken in seconds: ", difference)